# Analyse french stocks

In this tutorial, we will use the basic technics to analyse all french stock and try to find out which stock is worth for investment.

The objectives:
1. Find the top 20 stock with the best dividendYield of the last 5 years
2. Find the top 20 stock which are under evaluated ()
3. Find the top 20 stock which lost values in percentage.

## 1. Get all french stock symbols (ticker)

We can download the list of the stock symbols from this [site](https://www.abcbourse.com/download/libelles)



In [1]:
import pandas as pd
import yfinance as yf
import datetime
from pathlib import Path

In [2]:
data_dir = Path.cwd().parent / 'data'
filePath=  data_dir / "fr_stocks.csv"

stockSymbolDf = pd.read_csv(filePath.as_posix(), sep=";")
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF
1,FR0000031577,Virbac,VIRP
2,FR0000031775,Vicat,VCT
3,FR0000032278,Latecoere,LAT
4,FR0000032526,Guerbet,GBT


In [3]:
# we have 216 stock in our list
stockSymbolDf.shape

(216, 3)

We can notice that the ticker is not valid for yf, We need to add `.PA` to complete. For example, we need to convert `AF` to `AF.PA`

In [4]:
bad = yf.Ticker("AF")
print(bad.info)

{'exchange': 'YHD', 'quoteType': 'MUTUALFUND', 'symbol': 'AF', 'underlyingSymbol': 'AF', 'shortName': '411094', 'timeZoneFullName': 'America/New_York', 'timeZoneShortName': 'EST', 'uuid': '06fa567b-72d7-3b7e-bea7-46973fbbd1d5', 'gmtOffSetMilliseconds': -18000000, 'maxAge': 86400, 'trailingPegRatio': None}


In [5]:
good=yf.Ticker("AF.PA")
print(good.info)

{'address1': '7, rue du Cirque', 'city': 'Paris', 'zip': '75008', 'country': 'France', 'phone': '33 1 43 17 20 20', 'website': 'https://www.airfranceklm.com', 'industry': 'Airlines', 'industryKey': 'airlines', 'industryDisp': 'Airlines', 'sector': 'Industrials', 'sectorKey': 'industrials', 'sectorDisp': 'Industrials', 'longBusinessSummary': 'Air France-KLM SA, together with its subsidiaries, provides passenger and cargo transportation services and aircraft maintenance in Metropolitan France, Benelux, rest of Europe, and internationally. The company operates through Network, Maintenance, Transavia, and Other segments. It also offers airframe maintenance, electronic, mechanical, pneumatic, hydraulic, and other services. The company was founded in 1919 and is headquartered in Paris, France.', 'fullTimeEmployees': 78113, 'companyOfficers': [{'maxAge': 1, 'name': 'Mr. Benjamin M. Smith', 'age': 53, 'title': 'CEO & Director', 'yearBorn': 1971, 'fiscalYear': 2023, 'totalPay': 2169954, 'exerci

In [6]:
# convert
postfix=".PA"
stockSymbolDf['ticker'] = stockSymbolDf['ticker']+postfix
stockSymbolDf.head(5)

,ISIN,nom,ticker
0,FR0000031122,Air France - KLM,AF.PA
1,FR0000031577,Virbac,VIRP.PA
2,FR0000031775,Vicat,VCT.PA
3,FR0000032278,Latecoere,LAT.PA
4,FR0000032526,Guerbet,GBT.PA


## 2. Enrich the stock data frame

We need more information to judge a stock.

### 2.1 Which sector the enterprise works on ?

In [26]:


from typing import List


def get_stock_infos(stock_tickers:List[str],add_postfix:bool=False)->pd.DataFrame:
    """
    This function takes a list of stock tickers and returns a Pandas DataFrame which contains the basic information of the stock.
    The stock ticker has the following format: <stock_id>.PA. For example: ALO.PA: Alstom, MC.PA: LVMH
    :param add_postfix: If this option is enabled, a postfix ".PA" will be added to the stock ticker. This is for the ticker without .PA such as ALO, 
    :param stock_tickers: A list of stock tickers
    :return: 
    """
    # We don't need all information of a stock, below is a list of all important columns for me.
    key_columns = ["symbol", "industry", "sector", "longBusinessSummary","fullTimeEmployees","auditRisk","boardRisk","compensationRisk","shareHolderRightsRisk","overallRisk", "dividendRate","dividendYield",
                   "payoutRatio","fiveYearAvgDividendYield", "enterpriseValue", "fiftyTwoWeekLow", "fiftyTwoWeekHigh","fiftyDayAverage","lastDividendValue","lastDividendDate","totalDebt","freeCashflow",
                   "operatingCashflow","earningsGrowth", "revenueGrowth", "grossMargins"]
    stock_infos:List[dict] = []

    # add stock dict into a list
    for stock_ticker in stock_tickers:
        if add_postfix:
            full_name = f"{stock_ticker}.PA"
        else:
            full_name = stock_ticker
        try:
            stock = yf.Ticker(full_name)
            if stock:
                stock_infos.append(stock.info)
        except Exception as e:
            print(f"The stock ticker {stock_ticker} is no longer valid. {e}")
    # convert the list of dict into pandas dataframe, then filter only the column that interest me
    pdf = pd.DataFrame(stock_infos)[key_columns]
    # use symbol column as index
    pdf.set_index("symbol", inplace=True)
    return pdf

In [14]:
tickerList = stockSymbolDf['ticker'].tolist()
print(tickerList)

['AF.PA', 'VIRP.PA', 'VCT.PA', 'LAT.PA', 'GBT.PA', 'TOUP.PA', 'ALTA.PA', 'JCQ.PA', 'ICAD.PA', 'BEN.PA', 'BLC.PA', 'LBIRD.PA', 'MTU.PA', 'SCHP.PA', 'AURE.PA', 'BOL.PA', 'MRN.PA', 'NEX.PA', 'ACA.PA', 'FII.PA', 'SOP.PA', 'MAU.PA', 'ATO.PA', 'TEP.PA', 'RMS.PA', 'AKW.PA', 'MMT.PA', 'CDA.PA', 'DBG.PA', 'CGM.PA', 'HCO.PA', 'UBI.PA', 'TFI.PA', 'BOI.PA', 'PERR.PA', 'EXA.PA', 'VRAP.PA', 'DKUPL.PA', 'AUB.PA', 'BON.PA', 'COV.PA', 'PEUG.PA', 'GRVO.PA', 'LACR.PA', 'GLO.PA', 'GUI.PA', 'PIG.PA', 'ATE.PA', 'VAC.PA', 'SAF.PA', 'IPS.PA', 'BIG.PA', 'SII.PA', 'ASY.PA', 'PRC.PA', 'ORAP.PA', 'EOS.PA', 'DEC.PA', 'AI.PA', 'CA.PA', 'TTE.PA', 'OR.PA', 'AC.PA', 'EN.PA', 'QDT.PA', 'SAN.PA', 'CS.PA', 'BN.PA', 'ES.PA', 'RI.PA', 'NK.PA', 'BB.PA', 'MC.PA', 'RF.PA', 'FRVIA.PA', 'MF.PA', 'SW.PA', 'HO.PA', 'KER.PA', 'EL.PA', 'SK.PA', 'LI.PA', 'SU.PA', 'VIE.PA', 'POM.PA', 'SGO.PA', 'CAP.PA', 'DG.PA', 'CO.PA', 'VIV.PA', 'MMB.PA', 'RCO.PA', 'FGR.PA', 'PUB.PA', 'CRI.PA', 'GLE.PA', 'BNP.PA', 'ERA.PA', 'RNO.PA', 'ORA.PA', 'ORP

In [15]:
tickerDf = get_stock_infos(tickerList)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SII.PA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SII.PA&crumb=n1EpcZvZSOq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/POM.PA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=POM.PA&crumb=n1EpcZvZSOq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ORP.PA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ORP.PA&crumb=n1EpcZvZSOq
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SESL.PA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDoma

In [17]:

tickerDf.head(5)

,industry,sector,longBusinessSummary,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendRate,...,fiftyTwoWeekHigh,fiftyDayAverage,lastDividendValue,lastDividendDate,totalDebt,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins
symbol,,,,,,,,,,,,,,,,,,,,,
AF.PA,Airlines,Industrials,"Air France-KLM SA, together with its subsidiar...",78113.0,8.0,3.0,9.0,10.0,7.0,NaN,...,11.7940,7.69204,2.925217,1.215994e+09,1.351100e+10,-1.665000e+09,1.827000e+09,-0.517,0.037,0.18016
VIRP.PA,Drug Manufacturers - General,Healthcare,Virbac SA manufactures and sells a range of pr...,5459.0,10.0,8.0,7.0,9.0,8.0,1.32,...,400.0000,316.52000,1.320000,1.719274e+09,4.070020e+08,2.675250e+07,1.387030e+08,0.277,0.151,0.66660
VCT.PA,Building Materials,Basic Materials,"Vicat S.A., together with its subsidiaries, en...",9955.0,10.0,9.0,7.0,8.0,9.0,2.00,...,41.5500,37.08500,2.000000,1.714349e+09,2.107786e+09,1.179301e+08,5.701770e+08,0.105,0.013,0.35156
LAT.PA,Aerospace & Defense,Industrials,"Latécoère S.A. designs, manufactures, assemble...",5918.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0178,0.00882,0.048234,1.214352e+09,4.390530e+08,-5.025575e+07,-5.124300e+07,NaN,0.452,0.34669
GBT.PA,Medical Devices,Healthcare,Guerbet SA engages in the development and mark...,2920.0,6.0,7.0,8.0,7.0,8.0,0.50,...,40.2500,26.37100,0.500000,1.719792e+09,4.039190e+08,8.417000e+06,7.042500e+07,4.438,0.119,0.74818


In [18]:
outPath= data_dir / "fr_stock_info.csv"

In [19]:
tickerDf.to_csv(outPath,index=True)

## 3. Get the top 20 stock of dividend yield

In [21]:
sortByDiv=tickerDf.sort_values(by="dividendYield",ascending=False)
sortByDiv.head(20)

,industry,sector,longBusinessSummary,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendRate,...,fiftyTwoWeekHigh,fiftyDayAverage,lastDividendValue,lastDividendDate,totalDebt,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins
symbol,,,,,,,,,,,,,,,,,,,,,
ICAD.PA,REIT - Diversified,Real Estate,Icade is a full-service real estate company wi...,1035.0,7.0,7.0,2.0,1.0,4.0,4.84,...,32.000,22.331200,2.42,1.719878e+09,4.763100e+09,3.516850e+09,2.653000e+08,NaN,-0.002,0.36465
ASY.PA,Engineering & Construction,Industrials,Assystem S.A. provides engineering and infrast...,7500.0,7.0,9.0,9.0,8.0,9.0,8.00,...,61.200,39.293000,4.50,1.720570e+09,9.670000e+07,1.071875e+08,4.700000e+07,-0.746,0.067,0.29130
MMT.PA,Broadcasting,Communication Services,Métropole Télévision S.A. provides a range of ...,1558.0,4.0,8.0,8.0,6.0,8.0,1.25,...,14.960,11.463200,1.25,1.714435e+09,1.371000e+08,1.100625e+08,2.408000e+08,-0.188,0.057,0.30892
VIV.PA,Entertainment,Communication Services,"Vivendi SE operates as an entertainment, media...",39200.0,9.0,5.0,9.0,10.0,7.0,0.25,...,11.170,3.463948,0.25,1.714435e+09,9.083000e+09,1.763625e+09,1.337000e+09,-0.074,0.927,0.47645
MERY.PA,REIT - Retail,Real Estate,Mercialys is one of France's leading real esta...,161.0,4.0,4.0,3.0,1.0,3.0,0.99,...,12.740,10.238800,0.99,1.714349e+09,1.197752e+09,-3.861012e+07,1.583340e+08,0.188,0.039,0.96892
ENGI.PA,Utilities - Diversified,Utilities,"ENGIE SA engages in the power, natural gas, an...",97000.0,5.0,4.0,2.0,10.0,4.0,1.43,...,16.390,15.389200,1.43,1.714608e+09,4.938000e+10,3.753750e+08,1.256700e+10,NaN,-0.202,0.31299
BEN.PA,Recreational Vehicles,Consumer Cyclical,"Bénéteau S.A. designs, manufactures, and sells...",8000.0,10.0,7.0,7.0,8.0,7.0,0.73,...,14.380,8.626600,0.73,1.718150e+09,4.480040e+08,-5.374875e+07,8.137900e+07,-0.579,-0.315,0.59498
COFA.PA,Insurance - Reinsurance,Financial Services,"COFACE SA, through its subsidiaries, provides ...",4100.0,6.0,2.0,2.0,1.0,2.0,1.30,...,16.090,14.715800,1.30,1.716336e+09,3.645193e+09,4.719108e+08,3.114540e+08,0.062,0.022,0.63975
SCR.PA,Insurance - Reinsurance,Financial Services,"SCOR SE, together with its subsidiaries, provi...",3491.0,6.0,1.0,3.0,1.0,1.0,1.80,...,32.480,24.064000,1.80,1.716250e+09,3.214000e+09,-2.765000e+08,1.529000e+09,NaN,0.037,0.06883


The above dividendYield is the current year. In our case, it's 2023. If we want to get the historical value, we need to calculate by ourselves. The formula is pretty simple : `divYield = currentDiv / currentPrice`.

In [22]:
# we need a function which calculate the actual dividendYield
def getDividendYield(stockTicker:str, currentYear:int=None):
    """
    This function returns the dividendYield of a given year. If year is not provided, use the current year
    :param stockTicker:
    :type stockTicker:
    :param currentYear:
    :type currentYear:
    :return:
    :rtype:
    """
    stock = yf.Ticker(stockTicker)
    if not currentYear:
        currentYear=datetime.datetime.now().year

    currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
    currentPrice = stock.history(period="1d")["Close"][0]
    divYield = currentDiv / currentPrice
    print(f"The dividendYield of {stockTicker} at year {currentYear} is: {divYield}")

In [23]:
getDividendYield("NXI.PA", 2020)

The dividendYield of NXI.PA at year 2020 is: 0.14947682981268567


C:\Users\PLIU\AppData\Local\Temp\ipykernel_3104\1339861938.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
C:\Users\PLIU\AppData\Local\Temp\ipykernel_3104\1339861938.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  currentPrice = stock.history(period="1d")["Close"][0]


In [24]:
def getDivPerSharePeriodMean(stockTicker:str, period:int=5)->float:
    """
    This function returns the mean of the dividend per share over a given period. By default, the period is 5 years.
    :param stockTicker:
    :type stockTicker:
    :param period:
    :type period:
    :return:
    :rtype:
    """
    currentYear=datetime.datetime.now().year
    stock = yf.Ticker(stockTicker)
    total = 0
    for i in range(period):
        try:
            currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]
        except IndexError:
            currentDiv = 0
        print(f"The dividend of {stockTicker} at year {currentYear} is: {currentDiv}")
        total +=currentDiv
        currentYear -=1
    print(total)
    if total>0:
        divMean = total/period
    else:
        divMean = 0
    print(divMean)
    return divMean

In [25]:
getDivPerSharePeriodMean("NXI.PA")

The dividend of NXI.PA at year 2025 is: 0
The dividend of NXI.PA at year 2024 is: 0
The dividend of NXI.PA at year 2023 is: 2.5
The dividend of NXI.PA at year 2022 is: 2.5
The dividend of NXI.PA at year 2021 is: 2.0
7.0
1.4


C:\Users\PLIU\AppData\Local\Temp\ipykernel_3104\48149270.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  currentDiv = stock.dividends[stock.dividends.index.year == currentYear][0]


np.float64(1.4)

In [41]:
divMeanList = []
for ticker in tickerList:
    print(ticker)
    divMean = {"symbol": ticker,"divPerShare5YearMean": getDivPerSharePeriodMean(ticker)}
    divMeanList.append(divMean)

divMeanDf= pd.DataFrame(divMeanList)
divMeanDf.set_index("symbol",inplace=True)

divMeanDf.head(5)

AF.PA
The dividend of AF.PA at year 2023 is: 0
The dividend of AF.PA at year 2022 is: 0
The dividend of AF.PA at year 2021 is: 0
The dividend of AF.PA at year 2020 is: 0
The dividend of AF.PA at year 2019 is: 0
0
0
VIRP.PA
The dividend of VIRP.PA at year 2023 is: 1.32
The dividend of VIRP.PA at year 2022 is: 1.25
The dividend of VIRP.PA at year 2021 is: 0.75
The dividend of VIRP.PA at year 2020 is: 0
The dividend of VIRP.PA at year 2019 is: 0
3.3200000000000003
0.664
VCT.PA
The dividend of VCT.PA at year 2023 is: 1.65
The dividend of VCT.PA at year 2022 is: 1.65
The dividend of VCT.PA at year 2021 is: 1.5
The dividend of VCT.PA at year 2020 is: 1.5
The dividend of VCT.PA at year 2019 is: 1.5
7.8
1.56
LAT.PA
The dividend of LAT.PA at year 2023 is: 0
The dividend of LAT.PA at year 2022 is: 0
The dividend of LAT.PA at year 2021 is: 0
The dividend of LAT.PA at year 2020 is: 0
The dividend of LAT.PA at year 2019 is: 0
0
0
GBT.PA
The dividend of GBT.PA at year 2023 is: 0.5
The dividend of GB

,divPerShare5YearMean
symbol,
AF.PA,0.000
VIRP.PA,0.664
VCT.PA,1.560
LAT.PA,0.000
GBT.PA,0.720


In [44]:
stockDivSummary = pd.merge(sortByDiv,divMeanDf,on="symbol",how="inner")
stockDivSummary.head(20)

,shortName,longBusinessSummary,dividendYield,marketCap,beta,forwardPE,currentPrice,targetHighPrice,targetLowPrice,divPerShare5YearMean
symbol,,,,,,,,,,
NXI.PA,NEXITY,Nexity SA operates as a real estate company in...,0.1809,7.596621e+08,1.236563,6.408450,13.650,28.9,15.0,2.300000
ICAD.PA,ICADE,As a commercial property investor (portfolio w...,0.1310,2.495086e+09,1.179031,7.377575,32.240,40.0,40.0,2.196000
ALD.PA,ALD,ALD S.A. provides service leasing and vehicle ...,0.1215,7.408224e+09,0.953822,5.213018,8.810,20.1,15.0,0.744802
ALTA.PA,ALTAREA,Altarea is the French leader in low-carbon urb...,0.1199,1.728011e+09,1.083270,14.850087,84.200,103.0,101.0,9.891554
MERY.PA,MERCIALYS,Mercialys is one of France's leading real esta...,0.1191,7.414276e+08,1.571158,8.777778,7.900,12.0,7.5,0.682000
COFA.PA,COFACE,"COFACE SA, through its subsidiaries, provides ...",0.1177,1.930410e+09,0.880393,8.736486,12.930,17.2,13.0,0.872000
ENGI.PA,ENGIE,"ENGIE SA engages in the power, natural gas, an...",0.0965,3.603044e+10,0.997728,8.653488,14.884,21.2,14.2,0.790000
KOF.PA,KAUFMAN ET BROAD,Kaufman & Broad S.A. operates as a property de...,0.0958,5.229205e+08,1.055261,9.132841,24.750,35.0,24.5,2.090000
COV.PA,COVIVIO,"Thanks to its partnering history, its real est...",0.0925,4.003320e+09,1.460635,9.638554,40.000,62.0,44.0,4.100000


## Analyze CAC40.

In [33]:
filePath=  data_dir / "CAC40_2024.csv"

CAC40_symbol_pdf = pd.read_csv(filePath.as_posix(), sep=",")

In [35]:
cac40_tickers = CAC40_symbol_pdf['stock_id'].tolist()
print(cac40_tickers)

['MC', 'SU', 'TTE', 'SAN', 'AIR', 'AI', 'OR', 'SAF', 'EL', 'RMS', 'CS', 'BNP', 'DG', 'BN', 'SGO', 'ENGI', 'STLAP', 'LR', 'CAP', 'PUB', 'ML', 'DSY', 'RI', 'GLE', 'VIE', 'ORA', 'KER', 'STMPA', 'HO', 'ACA', 'BVI', 'AC', 'RNO', 'MT', 'URW', 'EDEN', 'CA', 'ERF', 'EN', 'TEP']


In [36]:
cac40_pdf = get_stock_infos(cac40_tickers,add_postfix=True)

In [37]:
cac40_pdf.head(5)

,industry,sector,longBusinessSummary,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendRate,...,fiftyTwoWeekHigh,fiftyDayAverage,lastDividendValue,lastDividendDate,totalDebt,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins
symbol,,,,,,,,,,,,,,,,,,,,,
MC.PA,Luxury Goods,Consumer Cyclical,"LVMH Moët Hennessy - Louis Vuitton, Société Eu...",192287.0,4.0,8.0,10.0,7.0,9.0,13.00,...,886.40,653.3360,5.50,1.733098e+09,4.085100e+10,9.772125e+09,1.893400e+10,-0.141,-0.013,0.68528
SU.PA,Specialty Industrial Machinery,Industrials,Schneider Electric S.E. engages in the energy ...,164640.0,10.0,3.0,6.0,7.0,6.0,3.50,...,273.00,246.6150,3.50,1.716854e+09,1.569600e+10,4.634875e+09,5.982000e+09,-0.070,0.031,0.42311
TTE.PA,Oil & Gas Integrated,Energy,"TotalEnergies SE, a multi-energy company, prod...",102579.0,4.0,4.0,1.0,7.0,3.0,3.16,...,70.11,54.5592,0.85,1.750291e+09,6.009100e+10,2.036875e+10,3.449700e+10,-0.648,-0.128,0.33842
SAN.PA,Drug Manufacturers - General,Healthcare,"Sanofi, a healthcare company, engages in the r...",86088.0,2.0,2.0,2.0,6.0,2.0,3.76,...,106.14,95.1640,3.76,1.715558e+09,2.375100e+10,NaN,NaN,0.115,0.115,0.69115
AIR.PA,Aerospace & Defense,Industrials,"Airbus SE, together with its subsidiaries, eng...",156569.0,5.0,1.0,2.0,7.0,1.0,1.80,...,172.78,158.0832,1.00,1.713226e+09,1.312000e+10,-1.258125e+09,4.969000e+09,0.216,0.053,0.15009


In [38]:
def get_top_dividendYield_stock(stock_df, top_n:int=20):
    """
    This function returns the top n dividendYield of a given stock.
    :param stock_df: 
    :param top_n: 
    :return: 
    """
    sortByDiv=stock_df.sort_values(by="dividendYield",ascending=False)
    return sortByDiv.head(top_n)

In [40]:
top_20=get_top_dividendYield_stock(cac40_pdf, 20)
top_20.head(20)

,industry,sector,longBusinessSummary,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendRate,...,fiftyTwoWeekHigh,fiftyDayAverage,lastDividendValue,lastDividendDate,totalDebt,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins
symbol,,,,,,,,,,,,,,,,,,,,,
STLAP.PA,Auto Manufacturers,Consumer Cyclical,"Stellantis N.V. engages in the design, enginee...",258275.0,10.0,3.0,5.0,6.0,4.0,1.55,...,27.345,12.57920,1.55,1.713744e+09,3.226100e+10,8.450000e+08,1.398100e+10,-0.455,-0.136,0.18143
ENGI.PA,Utilities - Diversified,Utilities,"ENGIE SA engages in the power, natural gas, an...",97000.0,5.0,4.0,2.0,10.0,4.0,1.43,...,16.390,15.38920,1.43,1.714608e+09,4.938000e+10,3.753750e+08,1.256700e+10,NaN,-0.202,0.31299
ACA.PA,Banks - Regional,Financial Services,"Crédit Agricole S.A. provides retail, corporat...",78823.0,3.0,7.0,6.0,2.0,6.0,1.05,...,15.925,13.46680,1.05,1.716941e+09,4.986000e+11,NaN,NaN,-0.055,0.024,NaN
BNP.PA,Banks - Regional,Financial Services,BNP Paribas SA provides various banking and fi...,183000.0,1.0,4.0,1.0,1.0,2.0,4.60,...,73.080,60.21000,4.60,1.716250e+09,6.303630e+11,NaN,NaN,0.154,0.034,NaN
ORA.PA,Telecom Services,Communication Services,"Orange S.A. provides fixed telephony, mobile t...",137094.0,1.0,4.0,1.0,6.0,3.0,0.72,...,11.115,9.87064,0.30,1.733184e+09,4.317300e+10,4.090000e+08,1.103400e+10,-0.067,0.025,0.38692
CA.PA,Grocery Stores,Consumer Defensive,Carrefour SA engages in the operation of store...,305309.0,5.0,6.0,8.0,6.0,7.0,0.87,...,16.920,13.78340,0.87,1.716854e+09,2.091700e+10,2.204125e+09,4.572000e+09,-0.969,-0.002,0.18280
EN.PA,Engineering & Construction,Industrials,"Bouygues SA, together with its subsidiaries, o...",201500.0,8.0,7.0,7.0,10.0,7.0,1.90,...,38.260,29.10820,1.90,1.714435e+09,1.491700e+10,2.838375e+09,5.725000e+09,0.137,0.015,0.54913
TTE.PA,Oil & Gas Integrated,Energy,"TotalEnergies SE, a multi-energy company, prod...",102579.0,4.0,4.0,1.0,7.0,3.0,3.16,...,70.110,54.55920,0.85,1.750291e+09,6.009100e+10,2.036875e+10,3.449700e+10,-0.648,-0.128,0.33842
CS.PA,Insurance - Diversified,Financial Services,"AXA SA, through its subsidiaries, provides ins...",94705.0,5.0,2.0,1.0,6.0,1.0,1.98,...,37.410,34.44180,1.98,1.714435e+09,6.077800e+10,-2.974375e+09,8.660000e+09,0.060,0.081,0.15099


In [41]:
def get_top_potential_stock(stock_df, top_n:int=20):
    """
    This function returns the top n potential stock of a given stock. We use the (last 52 week high - 50 day average price )/ 52 week high. If the number is positive we keep, 
    :param stock_df: 
    :param top_n: 
    :return: 
    """
    stock_df["history_price_diff"]=(stock_df["fiftyTwoWeekHigh"]-stock_df["fiftyDayAverage"])/stock_df["fiftyTwoWeekHigh"]
    stock_df = stock_df.sort_values(by="history_price_diff",ascending=False)
    return stock_df

In [42]:
top_potential = get_top_potential_stock(top_20, 20)
top_potential.head(20)

,industry,sector,longBusinessSummary,fullTimeEmployees,auditRisk,boardRisk,compensationRisk,shareHolderRightsRisk,overallRisk,dividendRate,...,fiftyDayAverage,lastDividendValue,lastDividendDate,totalDebt,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,history_price_diff
symbol,,,,,,,,,,,,,,,,,,,,,
STLAP.PA,Auto Manufacturers,Consumer Cyclical,"Stellantis N.V. engages in the design, enginee...",258275.0,10.0,3.0,5.0,6.0,4.0,1.55,...,12.57920,1.55,1.713744e+09,3.226100e+10,8.450000e+08,1.398100e+10,-0.455,-0.136,0.18143,0.539982
KER.PA,Luxury Goods,Consumer Cyclical,Kering SA manages the development of a series ...,46444.0,2.0,5.0,2.0,7.0,4.0,11.50,...,237.22800,2.00,1.736813e+09,1.936300e+10,1.545500e+09,4.308000e+09,-0.509,-0.110,0.75370,0.459124
EDEN.PA,Credit Services,Financial Services,Edenred SE provides digital platform for servi...,11008.0,3.0,4.0,8.0,4.0,5.0,1.10,...,31.32280,1.10,1.717978e+09,5.109000e+09,7.396250e+08,1.093000e+09,0.171,0.176,0.40984,0.450863
TEP.PA,Specialty Business Services,Industrials,"Teleperformance SE, together with its subsidia...",360980.0,1.0,3.0,5.0,5.0,3.0,3.85,...,85.67040,3.85,1.716854e+09,5.460000e+09,1.039875e+09,1.556000e+09,0.052,0.282,0.31119,0.399647
RI.PA,Beverages - Wineries & Distilleries,Consumer Defensive,Pernod Ricard SA produces and sells wines and ...,19557.0,4.0,4.0,7.0,7.0,6.0,4.70,...,107.98600,2.35,1.732493e+09,1.363400e+10,6.972500e+08,1.727000e+09,NaN,-0.003,0.60140,0.343750
EN.PA,Engineering & Construction,Industrials,"Bouygues SA, together with its subsidiaries, o...",201500.0,8.0,7.0,7.0,10.0,7.0,1.90,...,29.10820,1.90,1.714435e+09,1.491700e+10,2.838375e+09,5.725000e+09,0.137,0.015,0.54913,0.239200
TTE.PA,Oil & Gas Integrated,Energy,"TotalEnergies SE, a multi-energy company, prod...",102579.0,4.0,4.0,1.0,7.0,3.0,3.16,...,54.55920,0.85,1.750291e+09,6.009100e+10,2.036875e+10,3.449700e+10,-0.648,-0.128,0.33842,0.221806
CA.PA,Grocery Stores,Consumer Defensive,Carrefour SA engages in the operation of store...,305309.0,5.0,6.0,8.0,6.0,7.0,0.87,...,13.78340,0.87,1.716854e+09,2.091700e+10,2.204125e+09,4.572000e+09,-0.969,-0.002,0.18280,0.185378
BNP.PA,Banks - Regional,Financial Services,BNP Paribas SA provides various banking and fi...,183000.0,1.0,4.0,1.0,1.0,2.0,4.60,...,60.21000,4.60,1.716250e+09,6.303630e+11,NaN,NaN,0.154,0.034,NaN,0.176108



# interesting stock

- STLAP : high div_yield, stock price **low** compare to history. But automobile has high risk because of Trump terrif
- ENGI:  high div_yield, stock price **high** compare to history, 
- ACA: high div_yield, stock price **high** compare to history, 
